# Radioactive Decay
We will solve numerically the equation for radioactive decay of isotope A to isotope B and compare the numerical result to its analytical counterpart. We will use Euler's method.

As a second step we will generalize the solver to deal with three elements, where isotope A decays to isotope B and isotope B decays to isotope C. 

In [15]:
# Import some libraries from python and set some defaults
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.image as mpimg
import sys
import os
import time
sys.path.append(os.getcwd())
# from scipy.interpolate import CubicSpline
import pickle
import copy
plt.rcParams.update({'font.size': 17})
params = {'axes.labelsize': 16, 'axes.titlesize': 16}
plt.rcParams.update(params)
rc('animation', html='html5')


In [18]:
# get the base file
import requests
from pathlib import Path
if Path("DEq_Solver.py").is_file():
    print("DEq_Solver already exist")
else:
    print("Downloading the DEq_solver")
    request = requests.get("https://raw.githubusercontent.com/keithonpy/Scientific_Computing/main/Base/DEq_Solver.py")
    with open("DEq_Solver.py", "wb") as f:
        f.write(request.content)
print("Deq_Solver imported")
from DEq_Solver import DEq_Solver

DEq_Solver already exist
Deq_Solver imported


## Euler's method

Euler's method for solving the first order DE


$$\frac{{\rm d}x}{{\rm d}t} = f(x,t)$$ is



$$x^{n+1} = x^{n} + f(x^n,t^n)\,\Delta t$$.

We begin by implementing a routine, ``makeStep``, that implements this time integration, and test it in the special case where
$f(x,t)$ is a constant.


This is where we define the Euler solver. The class Euler Solver below is derived from the more general DEq_Solver class (see DEq_Solver.py in the same directory). This means that the resulting EulerSolver class features all the methods defined in DEq_Solver as well as the additional EulerSolver methods.

Implement the `makeStep(self)` function. Within this function you have access to 

- the current coordinates array as ``self.x``
- the current time as ``self.t``
- the derivative $dx/dt$ as ``self.kernel.dx_dt(self.x,self.t)``
- the timestep as ``self.delta_t``



the function ``makeStep`` has to update ``self.x`` and ``self.t`` using the Euler method for first order DE.

In [19]:
class EulerSolver(DEq_Solver):
    def __init__(self, kernel):
        self.kernel  = kernel
    def makeStep(self):
        ### update the position self.x, given dx_dt=self.kernel.dx_dt(self.x, self.t) and the timestep, self.delta_t
        ### and the timestep
        self.x +=  self.kernel.dx_dt(self.x, self.t) * self.delta_t
        self.t += self.delta_t

Two testing cases for the `makeStep()` routine

In [20]:
class ConstantSpeedKernel:
    def dx_dt(x,t):
        return 2.3
x0=np.array([0.0])
t0=0
t1=10
delta_t=0.1
d=EulerSolver(ConstantSpeedKernel)

d.initialise(x0,t0,t1,delta_t)

# test 20 steps
for i in range(20):
    d.makeStep()
assert np.isclose(d.x,np.array([ 4.6]))

In [21]:
class testKernel:
    def dx_dt(x,t):
        return sum(x)*np.ones_like(x)
    
d=EulerSolver(testKernel)
x0=np.array([1.0,2.0,3.0])
t0=0
t1=10
delta_t=0.1
d.initialise(x0,t0,t1,delta_t)

for i in range(10):
    d.makeStep()

assert np.isclose(d.x,np.array([ 26.57169837,  27.57169837,  28.57169837])).all()

## Radioactive decay

We use the ``EulerSolver`` routine to calculate the radioactive decay of isotope A. 
The mathematical model describing the radioactive decay of A to B is 

$$  \frac{{\rm d}N_A}{{\rm d}t} = -\frac{N_A}{\tau} $$

We will use the solver above but we now need to provide the kernel for this 
specific case -- that is we have to provide the function $f$ in the "canonical form"

$$  \frac{{\rm d}x_i}{{\rm d}t} = f_i(x_i,t)$$


The method `dx_dt(self,x,t)` needs to be modified. 
 
In this case we know the analytical solution so we can compare our numerical result with the analytical formula that you need to implement in ``analytical``

In addition, in the ``__init__`` routine the half-life given by `hlife` needs to be converted into the time constant `tau`. 

All other input, plotting of results, etc. is already implemented in the following cells.

In [22]:
class Radioactive:
    def __init__(self,hlife):        
        # set self.tau, given the half-life, hlife = tau * ln(2)
        self.tau = hlife / np.log(2)

    def dx_dt(self,x,t):
        # return dx/dt, given x, t, and tau
        return - x/ self.tau
        
    def analytical(self,x0,t):
        # return the analytical solution for decay, given x0, t, and tau
        return x0 * np.exp(-t/self.tau)

    def relative_error(self,test,t0,t1):
        res = []
        for t,x in test:
            if len(res)==0:
                x0 = x
            res.append( (t, x/self.analytical(x0,t-t0)-1) )
        return res

Test cases for the radioactive decay